In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
folder_path = 'data_nasdaq_csv'

In [14]:
dataframes = []
company_names = []

In [4]:
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    Ticker = filename.split(".")[0]
    company_names.append(Ticker)
    if os.path.isfile(file_path) and file_path.endswith(".csv"):
        try:
            df = pd.read_csv(file_path)
            df = df.assign(Ticker=Ticker)  # add company name column
            if df['Close'].isnull().sum() == 0:
                dataframes.append(df)
        except pd.errors.ParserError as e:
            print(f"Error in file {file_path}: {e}. Skipping file.")
        except Exception as e:
            print(f"Error in file {file_path}: {e}. Skipping file.")


Error in file data_nasdaq_csv/LRCX.csv: Error tokenizing data. C error: Expected 7 fields in line 4926, saw 9
. Skipping file.


In [15]:
df_1 = dataframes[0] # Get the first dataframe
df_1.plot(x='Date', y='Close', label=df_1['Ticker'].iloc[0], figsize = (16,8))
plt.show()

IndexError: list index out of range

In [ ]:
df = pd.concat(dataframes)
df_copy = df.copy()
df_copy = df_copy.drop(columns = ['Low', 'High', 'Open', 'Adjusted Close', 'Volume'], axis = 1)

In [ ]:
# convert the Date column to datetime format
df_copy['Date'] = pd.to_datetime(df_copy['Date'], format='%d-%m-%Y')

# get the minimum date for each ticker
min_dates = df_copy.groupby('Ticker')['Date'].min()

# filter out tickers whose minimum date is after year 2010
tickers_to_drop = min_dates[min_dates >= pd.to_datetime('2020-01-01')].index
df_copy = df_copy[~df_copy['Ticker'].isin(tickers_to_drop)]

In [11]:
df_copy

,Date,Close,Ticker
0,1990-02-16,0.077257,CSCO
1,1990-02-20,0.079861,CSCO
2,1990-02-21,0.078125,CSCO
3,1990-02-22,0.078993,CSCO
4,1990-02-23,0.078559,CSCO
...,...,...,...
4296,2022-12-06,1.730000,BWEN
4297,2022-12-07,1.720000,BWEN
4298,2022-12-08,1.730000,BWEN
4299,2022-12-09,1.730000,BWEN


In [8]:
### Split the dataset into time windows to get data samples.

# df = data_appl
X_data = []
y_data = []
dates = []
window_size = 30 # Time window for data feature (size of 'number of days') to learn.

# Only get the open price (index 2 in the data frame)
for i in range(1, len(df) - window_size - 1):
    first = df.iloc[i, 0]
    data_feature = []
    data_label = []
    
    # Get a window_size time frame for data feature
    for j in range(window_size):
        data_feature.append(df.iloc[i + j, 0])
        
    # Next value is the label (price of the next day) to be predicted
    data_label.append(df.iloc[i + window_size, 0])
    
    # Append new data sample (feature and label) to X_data and y_data
    X_data.append(np.array(data_feature).reshape(window_size, 1))
    y_data.append(np.array(data_label))
    dates.append(df.iloc[i + window_size, :].name)


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tf.keras.models import Sequential
from tf.keras.layers import LSTM, Dropout, Dense

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(30, 1), activation='relu'))  # LSTM layer with 128 units
model.add(Dropout(0.2))  # Dropout layer for regularization
model.add(LSTM(64, activation='relu'))  # LSTM layer with 64 units
model.add(Dense(32, activation='relu'))  # Dense layer with 32 units and ReLU activation
model.add(Dense(1))  # Dense output layer with 7 units